## git

In [2]:
%cd /content/drive/MyDrive/RAG/Retrieval-Augmented-Generation

/content/drive/MyDrive/RAG/Retrieval-Augmented-Generation


In [4]:
!git config --global user.email "soroush.pasandideh80@gmail.com"
!git config --global user.name "Soroush Pasandideh"

In [5]:
!git add .
!git commit -m "add pipeline from transformers library"
!git push origin main

[main 0ac457d] add pipeline from transformers library
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite Retrieval Augmented Generation-RAG.ipynb (72%)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 1.04 KiB | 88.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/Soroush-Pasandideh/Retrieval-Augmented-Generation.git
   c94a3df..0ac457d  main -> main


## Install dependencies

In [ ]:
!pip install langchain
!pip install chromadb
!pip install sentence-transformers
!pip install pypdf
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/

In [ ]:
!pip install -U bitsandbytes
!pip install -U git+https://github.com/huggingface/peft.git
!pip install -U git+https://github.com/huggingface/accelerate.git
!pip install -U einops
!pip install -U safetensors
!pip install -U xformers
!pip install -U ctransformers[cuda]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.6 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-c6b52m1a
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-c6b52m1a
  Resolved https://github.com/huggingface/peft.git to commit f2b6d13f1dbc971c7653aa65e82822ea2d84bb38
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.12.1.dev0-py3-none-any.whl size=294558 sha256=0addbd862442363551caf54cccad0c4ac24082f58f65084dcb69cb0ad63ec14f
  Stored in directory: /tmp/pip-ephem-wheel-cache-v2hlig_l/wheels/d7/c7/de/1368fac8590e1b103ddc2ec2a28ad51d83aded1a3830e8a087
Successfully built peft
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-6115rxgv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/acc

## Imports

In [ ]:
from transformers import AutoTokenizer,AutoModelForCausalLM,AutoConfig
from time import time
import transformers
import torch

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## Load model and tokenizer

In [ ]:
model_checkpoint = 'meta-llama/Meta-Llama-3-8B-Instruct'
model_config = AutoConfig.from_pretrained(pretrained_model_name_or_path=model_checkpoint,
                                          trust_remote_code=True,
                                          max_new_tokens=1024)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_checkpoint,
                                             trust_remote_code=True,
                                             config=model_config,
                                             device_map='auto')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

## Set up a text generation pipeline

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

In [ ]:
pipeline = pipeline(task="text_generation",
                    model=model,
                    tokenizer=tokenizer,
                    torch_dtype=torch.float16,
                    max_length=3000,
                    device_map="auto")

In [ ]:
llm = HuggingFacePipeline(pipeline=pipeline)